In [24]:
import numpy as np
from tqdm.auto import tqdm
from icecream import ic
from collections import deque

In [25]:
GOAL_STATE = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])


In [26]:
# Define the movement directions: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [27]:
def print_state(state):
    """Prints the state of the puzzle."""
    for row in state:
        print(" ".join(map(str, row)))
    print()

def get_blank_position(state):
    """Returns the position (row, col) of the blank tile (0)."""
    return tuple(np.argwhere(state == 0)[0])

def is_goal(state):
    """Checks if the current state is the goal state."""
    return np.array_equal(state, GOAL_STATE)

def get_neighbors(state):
    """Generate all possible neighbor states by sliding the empty space."""
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    # Find the position of the empty space (0)
    r, c = np.where(state == 0)
    r, c = r[0], c[0]  # There will be only one zero

    for dr, dc in directions:
        new_r, new_c = r + dr, c + dc
        if 0 <= new_r < 3 and 0 <= new_c < 3:
            # Make a copy of the current state and swap the empty space with the neighbor
            new_state = state.copy()
            new_state[r, c], new_state[new_r, new_c] = new_state[new_r, new_c], new_state[r, c]
            neighbors.append(new_state)

    return neighbors


BFS ALGORITHM

In [28]:

from collections import deque

def bfs(start_state):
    """Solves the puzzle using the BFS algorithm."""
    # Queue to store (state, path) where path is the sequence of states leading to this state
    queue = deque([(start_state, [])])
    visited = set()
    total_actions = 0  # To count the number of actions (states) evaluated

    while queue:
        state, path = queue.popleft()

        total_actions += 1  # Increment the total actions evaluated

        # If we have reached the goal state, return the solution path
        if is_goal(state):
            return path + [state], total_actions

        # Mark the current state as visited
        visited.add(tuple(map(tuple, state)))  # Use tuple to make it hashable

        # Explore all neighbors
        for neighbor in get_neighbors(state):
            if tuple(map(tuple, neighbor)) not in visited:
                queue.append((neighbor, path + [state]))

    return None, total_actions  # Return None if no solution is found

if __name__ == "__main__":
    # Define the initial state (example: 3x3 puzzle)
    start_state = np.array([[1, 2, 3], [4, 5, 6], [0, 7, 8]])

    # Define the goal state
    goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    # Print the initial state
    print("Initial State:")
    print_state(start_state)

    # Print the goal state
    print("Goal State:")
    print_state(goal_state)

    # Solve the puzzle using BFS
    solution, total_actions = bfs(start_state)

    if solution:
        solution_quality = len(solution) - 1  # Number of moves in the solution
        efficiency = solution_quality / total_actions if total_actions != 0 else 0  # Quality vs Total Cost
        
        print("Solution found in", solution_quality, "moves.")
        for i, state in enumerate(solution):
            print(f"Step {i + 1}:")
            print_state(state)
        
        # Display the additional metrics
        print("\nSolution Quality :", solution_quality )
        print("Total Cost :", total_actions)
        print("Efficiency :", efficiency)
    else:
        print("No solution found.")

Initial State:
1 2 3
4 5 6
0 7 8

Goal State:
1 2 3
4 5 6
7 8 0

Solution found in 2 moves.
Step 1:
1 2 3
4 5 6
0 7 8

Step 2:
1 2 3
4 5 6
7 0 8

Step 3:
1 2 3
4 5 6
7 8 0


Solution Quality : 2
Total Cost : 7
Efficiency : 0.2857142857142857


DFS Algorithm

In [31]:
import numpy as np


def dfs(start_state):
    """Solves the puzzle using the DFS algorithm."""
    # Stack to store (state, path) where path is the sequence of states leading to this state
    stack = [(start_state, [])]
    visited = set()
    total_actions = 0  # To count the number of actions (states) evaluated

    while stack:
        state, path = stack.pop()

        total_actions += 1  # Increment the total actions evaluated

        # If we have reached the goal state, return the solution path
        if is_goal(state):
            return path + [state], total_actions

        # Mark the current state as visited
        visited.add(tuple(map(tuple, state)))  # Use tuple to make it hashable

        # Explore all neighbors (we explore them in reverse order to maintain DFS behavior)
        for neighbor in get_neighbors(state):
            if tuple(map(tuple, neighbor)) not in visited:
                stack.append((neighbor, path + [state]))

    return None, total_actions  # Return None if no solution is found

if __name__ == "__main__":
    # Define the initial state (example: 3x3 puzzle)
    start_state = np.array([[1, 2, 3], [4, 5, 6], [0, 7, 8]])

    # Define the goal state
    goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    # Print the initial state
    print("Initial State:")
    print_state(start_state)

    # Print the goal state
    print("Goal State:")
    print_state(goal_state)

    # Solve the puzzle using DFS
    solution, total_actions = dfs(start_state)

    if solution:
        solution_quality = len(solution) - 1  # Number of moves in the solution
        efficiency = solution_quality / total_actions if total_actions != 0 else 0  # Quality vs Total Cost
        
        print("Solution found in", solution_quality, "moves.")
        for i, state in enumerate(solution):
            print(f"Step {i + 1}:")
            print_state(state)
        
        # Display the additional metrics
        print("\nSolution Quality (Number of moves):", solution_quality)
        print("Total Cost (Actions evaluated):", total_actions)
        print("Efficiency (Quality / Cost):", efficiency)
    else:
        print("No solution found.")

Initial State:
1 2 3
4 5 6
0 7 8

Goal State:
1 2 3
4 5 6
7 8 0

Solution found in 2 moves.
Step 1:
1 2 3
4 5 6
0 7 8

Step 2:
1 2 3
4 5 6
7 0 8

Step 3:
1 2 3
4 5 6
7 8 0


Solution Quality (Number of moves): 2
Total Cost (Actions evaluated): 3
Efficiency (Quality / Cost): 0.6666666666666666


A* Algorithm

In [68]:
def manhatten_distance(state, goal_state):
    distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if state[i][j] != 0:
                x, y = np.where(goal_state == state[i][j])
                distance += abs(x[0] - i) + abs(y[0] - j)
    return distance

In [84]:
import heapq
import numpy as np

def A_star_algo(initial_state):
    # priority queue stores (f(n), g(n), state, path)
    class State: 
        def __init__(self, state, g, path):
            self.state = state  # Current state of the puzzle
            self.g = g  # Cost from the initial state to the current state (g(n))
            self.h = manhattan_distance(state, GOAL_STATE)  # Heuristic cost from current state to goal state (h(n))
            self.f = self.g + self.h  # Total cost of the current state (f(n)) = g(n) + h(n)
            self.path = path  # Path from the initial state to the current state
        
        # Python's heapq needs a way to compare objects. By defining __lt__ we can compare two states to store them in the priority queue
        def __lt__(self, other):  # Less than function to compare two states
            if self.f == other.f:  # if the total cost is the same, we compare the heuristic cost
                return self.h < other.h 
            return self.f < other.f

    # Initialize the frontier with the initial state
    initial_state = State(initial_state, 0, []) 
    frontier = []  # priority queue of states to be explored
    visited = set()  # states already evaluated (visited nodes)
    
    # Push the initial state to the priority queue 
    heapq.heappush(frontier, initial_state) 
    nodes_expanded = 0
    
    while frontier:
        current = heapq.heappop(frontier)  # removes and returns the state with the lowest f(n) from the priority queue
        nodes_expanded += 1
        
        # Check if we've reached the goal
        if is_goal(current.state):
            quality = len(current.path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {nodes_expanded} states evaluated")
            print(f"Efficiency: {quality / nodes_expanded:.4f}")
            return current.path
        
        # Skip if already explored
        state_bytes = current.state.tobytes()
        if state_bytes in visited:
            continue
        
        visited.add(state_bytes)
        
        # Generate all possible actions (states) from the current state and add them to the frontier
        for next_state in get_neighbors(current.state):
            # Skip if already explored
            if next_state.tobytes() in visited:
                continue
            
            new_state = State(
                next_state,
                current.g + 1,
                current.path + [next_state]
            )
            
            heapq.heappush(frontier, new_state)
    
    print("No solution found!")
    return None

# Example usage
initial_state = np.array([[8, 2, 3], [1, 6, 4], [7, 0, 5]])
solution = A_star_algo(initial_state)

if solution:
    print("Solution found!")
    print_state(solution[-1])  # Display the final state of the puzzle


Solution Quality: 15 actions
Total Cost: 75 states evaluated
Efficiency: 0.2000
Solution found!
1 2 3
4 5 6
7 8 0

